In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

# 데이터 정리

In [2]:
stocks = pd.read_csv('./trade_train_set/stocks.csv')
trade_train = pd.read_csv('./trade_train_set/trade_train.csv')
stocks = stocks.drop(['index'], axis=1)
trade_train = trade_train.drop(['Unnamed: 0'], axis=1)

In [3]:
stocks_copy = stocks.copy()
stocks['datetime'] = stocks['기준일자'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
stocks.set_index(stocks['datetime'], inplace=True)

stocks = stocks.drop(['datetime','기준일자', '종목시가','종목고가','종목저가','종목종가'], 1)

In [4]:
numeric = stocks.select_dtypes('number').columns
non_num = stocks.columns.difference(numeric)
print(numeric)
print(non_num)

Index(['거래량', '거래금액_만원단위'], dtype='object')
Index(['20년7월TOP3대상여부', '시장구분', '종목명', '종목번호', '표준산업구분코드_대분류', '표준산업구분코드_소분류',
       '표준산업구분코드_중분류'],
      dtype='object')


In [5]:
#종목별로 찾아서, 월별로 묶음. 반복되는 데이터(종목코드 등)는 그대로 유지.

d = {**{x: 'mean' for x in numeric}, **{x: 'first' for x in non_num}}
monthly = None

for i, name in enumerate(stocks['종목명'].unique()):
    tmp =stocks[stocks['종목명'] == name].resample('M').agg(d)
    
    if i == 0:
        monthly = tmp
    else:
        monthly= pd.concat([monthly, tmp])

        
#평균가격을 계산
monthly['평균가격']= monthly['거래금액_만원단위']/monthly['거래량'] * 10000

#날짜를 인덱스에서 제거
monthly.reset_index(level=0, inplace=True)

#기준연월로 변경
monthly['기준년월'] = monthly['datetime'].dt.strftime('%Y%m')
monthly['기준년월'] = pd.to_numeric(monthly['기준년월'])

#datetime 삭제
monthly = monthly.drop(['datetime'], axis = 1)

monthly.to_csv("./month_stock.csv", mode='w', header=True, index=False)

In [6]:
# trade에 월별 stocks데이터를 합침.
total = pd.merge(trade_train, monthly, on=['기준년월', '종목번호'])
total.to_csv("./total_data.csv", mode='w', header=True, index=False)

# 훈련데이터 생성

In [7]:
data = total[total['20년7월TOP3대상여부'] == 'Y']

data = data.drop(['그룹내_매수여부', '그룹내_매도여부', '평균매수수량', '평균매도수량', 
                  '매수가격_중앙값', '매도가격_중앙값', '20년7월TOP3대상여부', '종목명'], axis=1)

In [8]:
a = trade_train[['기준년월', '그룹번호', '종목번호', '매수고객수']].groupby(['기준년월', '그룹번호',
                                                            '종목번호']).sum().sort_values(by = [
    '기준년월','그룹번호','매수고객수'], ascending=[True, True, False]).reset_index(drop=False)

a = a.groupby(['기준년월', '그룹번호']).head(3)

prev_month_list = list(a[a['기준년월'] == 202006]['종목번호'])

data['직전달TOP3여부'] = 0
for i in range(data.shape[0]):
    if data.iloc[i,3] in prev_month_list:
        data.iloc[i,-1] = 1

In [9]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
def dummy(data,col):
    lab=LabelEncoder()
    aa=lab.fit_transform(data[col]).reshape(-1,1)
    ohe=OneHotEncoder(sparse=False)
    column_names=[col+'_'+ str(i) for i in lab.classes_]
    return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [10]:
categ_columns = ['그룹번호', '종목번호','표준산업구분코드_대분류', '표준산업구분코드_중분류', 
                 '표준산업구분코드_소분류', '시장구분']
n_columns = ['기준년월','그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', '직전달TOP3여부']

df = data[n_columns]
df.index = range(df.shape[0])

In [11]:
lab = LabelEncoder()
for col in categ_columns:
    temp_df = pd.DataFrame(lab.fit_transform(data[col]))
    temp_df.columns = [col]
    df = pd.concat([df, temp_df], axis=1)

df.to_csv('./df.csv',mode='w', header=True, index=False)

# 훈련데이터 템플릿 만들기

In [12]:
df_temp = pd.DataFrame()

기준년월 = pd.Series(df['기준년월'].unique())
그룹번호 = pd.Series(df['그룹번호'].unique())
종목번호 = pd.Series(df['종목번호'].unique())

aa = []
for k in 기준년월:
    for i in 그룹번호:
        for j in 종목번호:
            aa.append([k,i,j])
        
aa = pd.DataFrame(data=aa, columns=['기준년월','그룹번호', '종목번호'])        
df_temp[['기준년월', '그룹번호', '종목번호']] = aa[['기준년월', '그룹번호', '종목번호']]

In [13]:
df_temp = pd.merge(df_temp, df[['종목번호', '직전달TOP3여부', '표준산업구분코드_대분류', '표준산업구분코드_중분류', 
                 '표준산업구분코드_소분류', '시장구분']].drop_duplicates(), on='종목번호')

df_temp = pd.merge(df_temp, df[['그룹번호','그룹내고객수']].drop_duplicates(), on='그룹번호')

df_temp = pd.merge(df_temp, df[['매수고객수','기준년월', '그룹번호', '종목번호', '거래량', 
                                '거래금액_만원단위', '평균가격']], 
        on = ['기준년월', '그룹번호', '종목번호'], how='outer')

df_temp = df_temp.fillna(0)


df_temp = df_temp[['기준년월', '그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', 
                   '직전달TOP3여부', '그룹번호', '종목번호', '표준산업구분코드_대분류', '표준산업구분코드_중분류', 
                   '표준산업구분코드_소분류', '시장구분']]

In [14]:
# 테스트 데이터 표준화를 위한 정보 저장
dat_mean = df_temp[['거래금액_만원단위', '평균가격', '거래량']].mean(axis=0)
dat_std = df_temp[['거래금액_만원단위', '평균가격', '거래량']].std()

# 훈련데이터 표준화
from sklearn.preprocessing import StandardScaler
df_temp[['거래금액_만원단위', '평균가격', '거래량']] = StandardScaler(
).fit_transform(df_temp[['거래금액_만원단위', '평균가격', '거래량']])

In [15]:
df_temp = df_temp[['기준년월', '그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', '직전달TOP3여부', '그룹번호',
       '종목번호', '표준산업구분코드_대분류', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']]

df_temp.to_csv('./df_temp.csv',mode='w', header=True, index=False)

# 예측 템플릿 (테스트 데이터)

In [16]:
# 예측 템플릿 만들기
그룹번호 = pd.Series(df['그룹번호'].unique())
종목번호 = pd.Series(df['종목번호'].unique())


temp = []
for i in 그룹번호:
    for j in 종목번호:
        temp.append([i, j])

                                
temp = pd.DataFrame(data=temp, columns = ['그룹번호', '종목번호'])

In [17]:
a = trade_train[['그룹번호','그룹내고객수']].groupby('그룹번호').first().reset_index(drop=False)
a['그룹번호'] = lab.fit_transform(a['그룹번호'])

temp = pd.merge(temp, a, on='그룹번호')

temp = pd.merge(temp, df[['표준산업구분코드_대분류','표준산업구분코드_중분류', '표준산업구분코드_소분류', 
                          '종목번호', '직전달TOP3여부', '시장구분']].drop_duplicates(), on='종목번호')

In [18]:
a = stocks_copy[stocks_copy['기준일자'] > 20200700].groupby(['종목번호']).sum().reset_index(drop=False)[[
    '종목번호', '거래금액_만원단위', '거래량']]
a['종목번호'] = lab.fit_transform(a['종목번호'])

a['종목번호'] = lab.fit_transform(a['종목번호'])

a['거래금액_만원단위'] = a['거래금액_만원단위']
a['평균가격'] = (a['거래금액_만원단위'] / a['거래량']) * 10000

In [19]:
temp = pd.merge(temp, a, on='종목번호')

temp['거래금액_만원단위'] = (temp['거래금액_만원단위'] - dat_mean[0]) / dat_std[0]
temp['평균가격'] = (temp['평균가격'] - dat_mean[1]) / dat_std[1]
temp['거래량'] = (temp['거래량'] - dat_mean[2]) / dat_std[2]

temp['기준년월'] = 202007

In [20]:
temp = temp[['기준년월', '그룹내고객수','거래량','거래금액_만원단위','평균가격','직전달TOP3여부', '그룹번호', 
      '종목번호','표준산업구분코드_대분류', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']]

temp.to_csv('./template.csv',mode='w', header=True, index=False)

# 그룹번호 & 종목번호 dictionary 만들기

In [21]:
lab1 = LabelEncoder()
lab2 = LabelEncoder()
lab1.fit_transform(data['그룹번호'])
lab2.fit_transform(data['종목번호'])

df_dict = pd.DataFrame()
df_dict['종목번호'] = temp['종목번호']
df_dict['종목번호_원본'] = 0
df_dict
for i in range(df_dict.shape[0]):
    df_dict['종목번호_원본'][i] =  lab2.inverse_transform([df_dict['종목번호'][i]])[0]
        

<ipython-input-21-303e9d4fe94c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict['종목번호_원본'][i] =  lab2.inverse_transform([df_dict['종목번호'][i]])[0]


In [22]:
df_dict = df_dict.drop_duplicates()

df_dict = df_dict.sort_values(by = ['종목번호'], ascending=[True])
df_dict.index = range(df_dict.shape[0])

df_dict.to_csv('./종목변환.csv',mode='w', header=True, index=False)

In [23]:
df_dict2 = pd.DataFrame()
df_dict2['그룹번호'] = temp['그룹번호']
df_dict2['그룹번호_원본'] = 0

for i in range(df_dict2.shape[0]):
    df_dict2['그룹번호_원본'][i] =  lab1.inverse_transform([df_dict2['그룹번호'][i]])[0]

<ipython-input-23-8c94d5a11a52>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict2['그룹번호_원본'][i] =  lab1.inverse_transform([df_dict2['그룹번호'][i]])[0]


In [24]:
df_dict2 = df_dict2.drop_duplicates()

df_dict2 = df_dict2.sort_values(by = ['그룹번호'], ascending=[True])
df_dict2.index = range(df_dict2.shape[0])

df_dict2.to_csv('./그룹변환.csv',mode='w', header=True, index=False)

# 매수 연속성을 통한 점수(Score)화

In [25]:
answer_sheet = pd.read_csv('./trade_train_set/answer_sheet.csv')

group_list = list(set(trade_train['그룹번호']))
group_list = sorted(group_list)

In [26]:
df=pd.DataFrame()
tmp = 0
for group in group_list :
    for day in set(trade_train['기준년월']) :
        condition1 = trade_train['그룹번호'] == group
        condition2 = trade_train['기준년월'] == day

        # 각 그룹별 월별 상위 매수 5종목을 뽑아 5점만점에서 1점씩 차등분배
        df_ = trade_train[condition1 & condition2]
        df_ = df_.sort_values(by=['매수고객수','평균매수수량'], ascending =False)
        df_['점수'] = df_[:5]['매수고객수'].rank()
        df_ = df_[:5]
        if tmp == 0 :
            df = df_
        else : 
            df = pd.merge(df,df_, how='outer')
        tmp += 1
score_df = df.sort_values(by=['그룹번호','기준년월'], ascending=True)

In [27]:
# 그룹별로 루프를 돌려 기준년월과 종목번호를 행렬로 갖는 그룹별 매수 Score 데이터 형성
for group in group_list :
    group_df = score_df[score_df['그룹번호'] == group]
    check_df = pd.DataFrame(index=set(group_df['기준년월']), columns=set(group_df['종목번호']))
    for day in set(group_df['기준년월']) :
        for stock in set(group_df['종목번호']) : 
            condition1= group_df['기준년월'] == day
            condition2= group_df['종목번호'] == stock
            try : 
                check_df[stock][day] = group_df[condition1 & condition2]['점수'].values[0]
            except IndexError : 
                pass
    check_df = check_df.sort_index()
    check_df.to_csv('./group/'+group+'_score.csv', index=True)

# 그룹번호 & 종목번호를 통한 점수 컬럼 추가

### 훈련데이터

In [28]:
df_temp.index = df_temp['기준년월']
df_temp = df_temp.drop(['기준년월'], axis=1)

In [29]:
item_num = df_dict

time_dict = {}
for num, time in enumerate(sorted(set(df_temp.index))) : 
    time_dict[time] = num + 1
    
item_dict = dict(zip(item_num['종목번호_원본'], item_num['종목번호']))

group_num = pd.DataFrame(columns=['그룹명','그룹번호'])
group_num['그룹명'] = answer_sheet['그룹명']
group_num['그룹번호'] = range(0,48)
group_dict = dict(zip(group_num['그룹명'], group_num['그룹번호']))

In [30]:
score_list = []

for i in range(len(df_temp)) :
    try : 
        time = df_temp.iloc[i].name
        group = df_temp.iloc[i]['그룹번호']
        item = df_temp.iloc[i]['종목번호']

        serch_group = [key for key, value in group_dict.items() if value == group]
        serch_item = [key for key, value in item_dict.items() if value == item]

        path = './group/'+serch_group[0]+'_score.csv'
        score_df = pd.read_csv(path, index_col=0)
        score_df = score_df.fillna(0)
        score_df = score_df.sort_index(ascending = False)

        score_array = score_df[-time_dict[time]:][serch_item[0]]

        # 단순지수평활(simple exponential smoothing) 가중치
        a = 0.8
        weight = []
        for i in range(len(score_array)) : 
            if i == 0 : 
                alpha = a
                weight.append(alpha)
            else : 
                alpha = alpha*(1-a)
                weight.append(alpha)

        score = score_array.values@np.array(weight)
        if time == 201907 : 
            score_list.append(0)
            score_list.append(score)
        elif time == 202006 : 
            pass
        else : 
            score_list.append(score)    
    
    except KeyError : 
        score_list.append(0)

In [31]:
df_temp['점수'] = score_list
df_temp = df_temp.reset_index(drop=False)
df_temp.to_csv('./df_temp.csv', index=True)

### 테스트 데이터

In [32]:
group_list = sorted(list(set(trade_train['그룹번호'])))

path = './group/'+group_list[0]+'_score.csv'
score_df = pd.read_csv(path, index_col=0)

time_dict = {}
for num, time in enumerate(score_df.index) : 
    time_dict[time] = num + 1
    
item_dict = dict(zip(item_num['종목번호_원본'], item_num['종목번호']))

group_num = pd.DataFrame(columns=['그룹명','그룹번호'])
group_num['그룹명'] = answer_sheet['그룹명']
group_num['그룹번호'] = range(0,48)

group_dict = dict(zip(group_num['그룹명'], group_num['그룹번호']))

In [33]:
score_list = []

for i in range(len(temp)) :
    try : 
        time = temp.iloc[i].name
        group = temp.iloc[i]['그룹번호']
        item = temp.iloc[i]['종목번호']

        serch_group = [key for key, value in group_dict.items() if value == group]
        serch_item = [key for key, value in item_dict.items() if value == item]

        path = './group/'+serch_group[0]+'_score.csv'
        score_df = pd.read_csv(path, index_col=0)
        score_df = score_df.fillna(0)
        score_df = score_df.sort_index(ascending = False)

        # 단순지수평활(simple exponential smoothing) 가중치
        a = 0.8
        weight = []
        for i in range(len(score_df)) : 
            if i == 0 : 
                alpha = a
                weight.append(alpha)
            else : 
                alpha = alpha*(1-a)
                weight.append(alpha)

        score = score_df[serch_item[0]]@np.array(weight)
        score_list.append(score)    
    
    except KeyError : 
        score_list.append(0)

In [34]:
temp['점수'] = score_list
temp.to_csv('./temp.csv', index=True)

# 모델 훈련

In [35]:
X_train = df_temp.drop(['그룹내고객수','시장구분','표준산업구분코드_대분류', '거래금액_만원단위', '거래량', '평균가격', 
                     '직전달TOP3여부', '표준산업구분코드_중분류', '표준산업구분코드_소분류'], axis=1)

X_train_x = X_train.drop(['매수고객수'], axis = 1)
X_train_y = X_train['매수고객수']

In [36]:
from catboost import CatBoostRegressor

categ = [1,2]
cb_model = CatBoostRegressor(iterations=1300, learning_rate=0.02, depth=8,
                             eval_metric='RMSE', random_seed = 23,
                             bagging_temperature = 0.2, od_type='Iter', od_wait=100)

cb_model.fit(X_train_x, X_train_y, cat_features=categ, use_best_model=False, verbose=True)

0:	learn: 10.9111029	total: 81.8ms	remaining: 1m 46s
1:	learn: 10.7677461	total: 94.8ms	remaining: 1m 1s
2:	learn: 10.6254482	total: 113ms	remaining: 49s
3:	learn: 10.4883659	total: 129ms	remaining: 41.9s
4:	learn: 10.3536057	total: 144ms	remaining: 37.2s
5:	learn: 10.2207280	total: 158ms	remaining: 34s
6:	learn: 10.0923003	total: 174ms	remaining: 32.2s
7:	learn: 9.9647204	total: 191ms	remaining: 30.9s
8:	learn: 9.8391475	total: 205ms	remaining: 29.3s
9:	learn: 9.7143429	total: 216ms	remaining: 27.9s
10:	learn: 9.5940582	total: 229ms	remaining: 26.8s
11:	learn: 9.4783470	total: 240ms	remaining: 25.8s
12:	learn: 9.3640819	total: 252ms	remaining: 25s
13:	learn: 9.2565348	total: 264ms	remaining: 24.3s
14:	learn: 9.1471619	total: 276ms	remaining: 23.6s
15:	learn: 9.0478495	total: 292ms	remaining: 23.4s
16:	learn: 8.9429747	total: 308ms	remaining: 23.3s
17:	learn: 8.8409084	total: 328ms	remaining: 23.4s
18:	learn: 8.7390926	total: 342ms	remaining: 23.1s
19:	learn: 8.6397836	total: 354ms	rem

162:	learn: 4.4194863	total: 2.59s	remaining: 18.1s
163:	learn: 4.4126534	total: 2.61s	remaining: 18.1s
164:	learn: 4.4076832	total: 2.62s	remaining: 18.1s
165:	learn: 4.4018913	total: 2.64s	remaining: 18.1s
166:	learn: 4.3953458	total: 2.67s	remaining: 18.1s
167:	learn: 4.3900456	total: 2.69s	remaining: 18.1s
168:	learn: 4.3802252	total: 2.71s	remaining: 18.1s
169:	learn: 4.3699420	total: 2.73s	remaining: 18.1s
170:	learn: 4.3608450	total: 2.75s	remaining: 18.1s
171:	learn: 4.3527074	total: 2.77s	remaining: 18.1s
172:	learn: 4.3447583	total: 2.79s	remaining: 18.1s
173:	learn: 4.3368237	total: 2.8s	remaining: 18.1s
174:	learn: 4.3294323	total: 2.82s	remaining: 18.1s
175:	learn: 4.3221558	total: 2.84s	remaining: 18.2s
176:	learn: 4.3137653	total: 2.86s	remaining: 18.1s
177:	learn: 4.3068164	total: 2.88s	remaining: 18.1s
178:	learn: 4.2986422	total: 2.89s	remaining: 18.1s
179:	learn: 4.2901155	total: 2.91s	remaining: 18.1s
180:	learn: 4.2828356	total: 2.92s	remaining: 18.1s
181:	learn: 4

336:	learn: 3.8056156	total: 5.05s	remaining: 14.4s
337:	learn: 3.8052748	total: 5.06s	remaining: 14.4s
338:	learn: 3.8048629	total: 5.07s	remaining: 14.4s
339:	learn: 3.8029773	total: 5.09s	remaining: 14.4s
340:	learn: 3.8021469	total: 5.11s	remaining: 14.4s
341:	learn: 3.8002290	total: 5.12s	remaining: 14.3s
342:	learn: 3.7987698	total: 5.13s	remaining: 14.3s
343:	learn: 3.7980012	total: 5.14s	remaining: 14.3s
344:	learn: 3.7976231	total: 5.15s	remaining: 14.2s
345:	learn: 3.7970198	total: 5.15s	remaining: 14.2s
346:	learn: 3.7948751	total: 5.17s	remaining: 14.2s
347:	learn: 3.7943252	total: 5.17s	remaining: 14.2s
348:	learn: 3.7942017	total: 5.19s	remaining: 14.1s
349:	learn: 3.7919656	total: 5.2s	remaining: 14.1s
350:	learn: 3.7902571	total: 5.21s	remaining: 14.1s
351:	learn: 3.7885631	total: 5.22s	remaining: 14.1s
352:	learn: 3.7876201	total: 5.23s	remaining: 14s
353:	learn: 3.7872556	total: 5.24s	remaining: 14s
354:	learn: 3.7852678	total: 5.25s	remaining: 14s
355:	learn: 3.78521

504:	learn: 3.5979164	total: 7.27s	remaining: 11.4s
505:	learn: 3.5977055	total: 7.28s	remaining: 11.4s
506:	learn: 3.5975137	total: 7.29s	remaining: 11.4s
507:	learn: 3.5968962	total: 7.3s	remaining: 11.4s
508:	learn: 3.5949389	total: 7.32s	remaining: 11.4s
509:	learn: 3.5945547	total: 7.33s	remaining: 11.4s
510:	learn: 3.5935947	total: 7.34s	remaining: 11.3s
511:	learn: 3.5913423	total: 7.36s	remaining: 11.3s
512:	learn: 3.5911221	total: 7.37s	remaining: 11.3s
513:	learn: 3.5903739	total: 7.38s	remaining: 11.3s
514:	learn: 3.5878594	total: 7.39s	remaining: 11.3s
515:	learn: 3.5867268	total: 7.41s	remaining: 11.3s
516:	learn: 3.5855177	total: 7.42s	remaining: 11.2s
517:	learn: 3.5825991	total: 7.44s	remaining: 11.2s
518:	learn: 3.5821853	total: 7.46s	remaining: 11.2s
519:	learn: 3.5802535	total: 7.47s	remaining: 11.2s
520:	learn: 3.5786969	total: 7.49s	remaining: 11.2s
521:	learn: 3.5774502	total: 7.51s	remaining: 11.2s
522:	learn: 3.5770054	total: 7.52s	remaining: 11.2s
523:	learn: 3

665:	learn: 3.3858053	total: 9.53s	remaining: 9.07s
666:	learn: 3.3847368	total: 9.55s	remaining: 9.06s
667:	learn: 3.3828435	total: 9.56s	remaining: 9.04s
668:	learn: 3.3813029	total: 9.58s	remaining: 9.03s
669:	learn: 3.3803698	total: 9.59s	remaining: 9.02s
670:	learn: 3.3793873	total: 9.6s	remaining: 9s
671:	learn: 3.3788962	total: 9.61s	remaining: 8.98s
672:	learn: 3.3782865	total: 9.63s	remaining: 8.97s
673:	learn: 3.3776356	total: 9.64s	remaining: 8.96s
674:	learn: 3.3757460	total: 9.66s	remaining: 8.94s
675:	learn: 3.3745897	total: 9.67s	remaining: 8.92s
676:	learn: 3.3736858	total: 9.68s	remaining: 8.9s
677:	learn: 3.3731799	total: 9.69s	remaining: 8.89s
678:	learn: 3.3722862	total: 9.7s	remaining: 8.87s
679:	learn: 3.3719335	total: 9.71s	remaining: 8.85s
680:	learn: 3.3713482	total: 9.72s	remaining: 8.84s
681:	learn: 3.3687157	total: 9.75s	remaining: 8.84s
682:	learn: 3.3671810	total: 9.77s	remaining: 8.83s
683:	learn: 3.3668342	total: 9.79s	remaining: 8.82s
684:	learn: 3.3654

827:	learn: 3.2094943	total: 11.8s	remaining: 6.73s
828:	learn: 3.2089149	total: 11.8s	remaining: 6.72s
829:	learn: 3.2070874	total: 11.8s	remaining: 6.71s
830:	learn: 3.2066369	total: 11.9s	remaining: 6.69s
831:	learn: 3.2062568	total: 11.9s	remaining: 6.68s
832:	learn: 3.2057734	total: 11.9s	remaining: 6.66s
833:	learn: 3.2054021	total: 11.9s	remaining: 6.65s
834:	learn: 3.2050571	total: 11.9s	remaining: 6.63s
835:	learn: 3.2046785	total: 11.9s	remaining: 6.62s
836:	learn: 3.2042429	total: 11.9s	remaining: 6.6s
837:	learn: 3.2038297	total: 11.9s	remaining: 6.58s
838:	learn: 3.2034956	total: 12s	remaining: 6.57s
839:	learn: 3.2030699	total: 12s	remaining: 6.55s
840:	learn: 3.2026518	total: 12s	remaining: 6.54s
841:	learn: 3.2009849	total: 12s	remaining: 6.52s
842:	learn: 3.2005769	total: 12s	remaining: 6.5s
843:	learn: 3.2000138	total: 12s	remaining: 6.49s
844:	learn: 3.1996523	total: 12s	remaining: 6.48s
845:	learn: 3.1983956	total: 12s	remaining: 6.46s
846:	learn: 3.1969148	total: 1

989:	learn: 3.0862228	total: 13.8s	remaining: 4.33s
990:	learn: 3.0860938	total: 13.8s	remaining: 4.31s
991:	learn: 3.0857538	total: 13.8s	remaining: 4.3s
992:	learn: 3.0855335	total: 13.9s	remaining: 4.29s
993:	learn: 3.0853179	total: 13.9s	remaining: 4.27s
994:	learn: 3.0851068	total: 13.9s	remaining: 4.25s
995:	learn: 3.0849203	total: 13.9s	remaining: 4.24s
996:	learn: 3.0826800	total: 13.9s	remaining: 4.22s
997:	learn: 3.0821486	total: 13.9s	remaining: 4.21s
998:	learn: 3.0818198	total: 13.9s	remaining: 4.2s
999:	learn: 3.0808795	total: 13.9s	remaining: 4.18s
1000:	learn: 3.0805599	total: 14s	remaining: 4.17s
1001:	learn: 3.0796307	total: 14s	remaining: 4.16s
1002:	learn: 3.0789474	total: 14s	remaining: 4.14s
1003:	learn: 3.0783887	total: 14s	remaining: 4.13s
1004:	learn: 3.0780889	total: 14s	remaining: 4.11s
1005:	learn: 3.0761713	total: 14s	remaining: 4.1s
1006:	learn: 3.0743536	total: 14s	remaining: 4.09s
1007:	learn: 3.0741791	total: 14.1s	remaining: 4.07s
1008:	learn: 3.073637

1146:	learn: 2.9613851	total: 15.8s	remaining: 2.11s
1147:	learn: 2.9609685	total: 15.8s	remaining: 2.1s
1148:	learn: 2.9601385	total: 15.9s	remaining: 2.08s
1149:	learn: 2.9597262	total: 15.9s	remaining: 2.07s
1150:	learn: 2.9593184	total: 15.9s	remaining: 2.06s
1151:	learn: 2.9590301	total: 15.9s	remaining: 2.04s
1152:	learn: 2.9588247	total: 15.9s	remaining: 2.03s
1153:	learn: 2.9584223	total: 15.9s	remaining: 2.01s
1154:	learn: 2.9581425	total: 15.9s	remaining: 2s
1155:	learn: 2.9579410	total: 15.9s	remaining: 1.99s
1156:	learn: 2.9575439	total: 16s	remaining: 1.97s
1157:	learn: 2.9566531	total: 16s	remaining: 1.96s
1158:	learn: 2.9563433	total: 16s	remaining: 1.94s
1159:	learn: 2.9562949	total: 16s	remaining: 1.93s
1160:	learn: 2.9559160	total: 16s	remaining: 1.92s
1161:	learn: 2.9550169	total: 16s	remaining: 1.9s
1162:	learn: 2.9549699	total: 16s	remaining: 1.89s
1163:	learn: 2.9548109	total: 16.1s	remaining: 1.88s
1164:	learn: 2.9532302	total: 16.1s	remaining: 1.86s
1165:	learn:

# 결과 예측

In [37]:
temp = temp.drop(['그룹내고객수','시장구분','표준산업구분코드_대분류', '거래금액_만원단위', '거래량', '평균가격', 
                     '직전달TOP3여부', '표준산업구분코드_중분류', '표준산업구분코드_소분류'], axis = 1)

y_test = cb_model.predict(temp)
a = pd.DataFrame(y_test, columns=['매수고객수'])
result = pd.concat([temp, a], axis = 1)

out = result[['그룹번호', '종목번호', '매수고객수']].sort_values(by=['그룹번호','매수고객수'], 
                                                    ascending = [True, False])

result = out.groupby(['그룹번호']).head(3)
# result
out

,그룹번호,종목번호,매수고객수
144,0,17,59.432110
0,0,3,55.754539
192,0,18,43.841916
1200,0,55,6.457575
96,0,16,5.693051
...,...,...,...
6073,47,120,-0.017339
6121,47,10,-0.017339
6169,47,122,-0.017339
6217,47,81,-0.017339


In [38]:
group_name = df_dict2
stock_name = df_dict
#result
output = pd.DataFrame(columns=['그룹번호','Top1','Top2','Top3'])

top_list = []
top = []
count = 0
for idx, col in result.iterrows():
    if count % 3 == 0:
        count +=1
        top.append(group_name.iloc[col['그룹번호'].astype(int), 1])
        top.append(stock_name.iloc[col['종목번호'].astype(int), 1])
    elif count % 3 == 1:
        count +=1
        top.append(stock_name.iloc[col['종목번호'].astype(int), 1])
    else:
        top.append(stock_name.iloc[col['종목번호'].astype(int), 1])
        top_list.append(top)
        #clear
        top = []
        count = 0
        
for idx, top in enumerate(top_list):
    output.loc[idx] = top

output.to_csv("./output.csv", mode='w', header=True, index=False)
output

,그룹번호,Top1,Top2,Top3
0,MAD01,A005930,A000660,A005935
1,MAD02,A105560,A005930,A006800
2,MAD03,A019170,A003000,A007570
3,MAD04,A005930,A000660,A068270
4,MAD05,A035720,A068270,A035420
5,MAD06,A005930,A035720,A035420
6,MAD07,A096770,A010950,A005930
7,MAD08,A019170,A003000,A005930
8,MAD09,A035720,A005930,A051910
9,MAD10,A015760,A005935,A030200
